In [29]:
import pandas as pd
import tools as t

Предобработка тектов вакансий

Добавить сюда ID вакансий и key_skills

In [73]:
df_descr = pd.DataFrame(pd.read_json('deduplicated.json')[['id', 'description', 'key_skills']])

Удаление нерелевантных символов

In [76]:
not_to_split = ['a/b', 'ci/cd', 'pl/sql', 'pl/pg', 'tcp/ip', 'ui/ux', 'ux/ui', 'а/б', 'а/в', 'в/из']

In [85]:
df_descr['d_clean'] = df_descr['description'].apply(lambda x: t.clean_text(x, not_to_split))
df_descr['ks_clean'] = df_descr['key_skills'].apply(lambda x: t.clean_text(str(x), not_to_split))

Токенизация

In [88]:
from nltk import word_tokenize

In [89]:
df_descr['d_tokens'] = df_descr['d_clean'].apply(lambda x: word_tokenize(x, language = 'russian'))
df_descr['ks_tokens'] = df_descr['ks_clean'].apply(lambda x: word_tokenize(x, language = 'russian'))

Лемматизация (17814 текста обрабатывались 52 мин 26 сек)

In [90]:
df_descr['d_lemmatized'] = df_descr['d_tokens'].apply(lambda x: t.lemmatization(x))

In [91]:
df_descr['ks_lemmatized'] = df_descr['ks_tokens'].apply(lambda x: t.lemmatization(x))

Скачивание стоп-слов для русского языка и добавление новых стоп-слов

In [92]:
from nltk.corpus import stopwords
sw_to_add = ['хорошее', 'знание', 'знать', 'опыт', 'работы', 'организация', 'работать', 'ms', 'условия', 'умение', 'навык']
sw_to_add = t.lemmatization(sw_to_add)
stopwords = stopwords.words('russian')
stopwords = stopwords + sw_to_add

Удаление стоп-слов

In [95]:
df_descr['d_wo_sw'] = df_descr['d_lemmatized'].apply(lambda x: t.remove_stop_words(x, stopwords))
df_descr['ks_wo_sw'] = df_descr['ks_lemmatized'].apply(lambda x: t.remove_stop_words(x, stopwords))

In [97]:
df_descr.to_json('preprocessed_text.json')

In [31]:
df_descr = pd.read_json('preprocessed_text.json')

In [98]:
df_descr['d_full'] = df_descr['d_wo_sw'].apply(lambda x: ' '.join(x))
df_descr['ks_full'] = df_descr['ks_wo_sw'].apply(lambda x: ' '.join(x))

SKILL COUNT BASED ON DICTIONARY

Создаем датафрейм из id, полного описания вакансии и key_skills, добавляем столбец с пустыми списками (для полученных навыков) 

In [279]:
descr = pd.DataFrame({'id': df_descr['id'], 'd_full': df_descr['d_full'], 'ks_full': df_descr['ks_full']})
descr['extracted_skills'] = descr.apply(lambda _: [], axis = 1)

Создаем словарь навыков

In [280]:
df_dict = pd.read_excel('automatic generated dict.xlsx')['name'].to_list()
skills = dict.fromkeys(df_dict, 0)

Функция для подсчета частоты встечаемости навыков и вытаскивания навыков в отдельное поле

In [248]:
import regex as re

In [249]:
def skills_count(id):
    ind = descr.index[descr['id'] == id].tolist()[0]
    text = descr['d_full'][ind]
    ks = descr['ks_full'][ind]
    for skill in skills:
        if re.search(skill, ks) != None:
            descr['extracted_skills'][ind].append(skill)
            skills[skill] += 1
        elif re.search(skill, text) != None:
            descr['extracted_skills'][ind].append(skill)
            skills[skill] += 1

In [281]:
descr['id'].apply(lambda x: skills_count(x))

0        None
1        None
2        None
3        None
4        None
         ... 
50686    None
50687    None
50688    None
50690    None
50691    None
Name: id, Length: 20578, dtype: object

Сортировка

In [282]:
skills = dict(sorted(skills.items(), key=lambda item: item[1], reverse=True))

In [283]:
skills

{'sql': 7473,
 'excel': 6723,
 'erp': 5163,
 '1с': 5115,
 'высокий образование': 4961,
 'технический задание': 4320,
 'bpmn': 3836,
 'аналитический мышление': 3785,
 'ответственность': 3435,
 'анализ данные': 3379,
 'база данные': 3303,
 'английский язык': 3272,
 'uml': 3050,
 'powerpoint': 2911,
 'бизнес анализ': 2874,
 'больший объём информация': 2642,
 'office': 2496,
 'аналитический склад ум': 2453,
 'jira': 2397,
 'python': 2307,
 'rest': 2206,
 'системный анализ': 2165,
 'разработка технический задание': 2048,
 'confluence': 2017,
 'api': 2000,
 'технический документация': 1988,
 'постановка задача разработчик': 1958,
 'многозадачность': 1868,
 'проектный документация': 1826,
 'моделирование бизнес процесс': 1799,
 'описание бизнес процесс': 1776,
 'внимательность': 1766,
 'консультирование': 1761,
 'agile': 1631,
 'сводный таблица': 1556,
 'soap': 1485,
 'статистика': 1481,
 'управление проект': 1478,
 'visio': 1459,
 'power bi': 1430,
 'word': 1365,
 'коммуникабельность': 1362,